In [1]:
import os
import sys
import torch
import time
import numpy as np
import cv2
from yacs.config import CfgNode as CN

In [2]:
sys.path.append("../0.mc_utils/")

In [5]:
from openpose.op_formats import NPMapOP25bToSMPL24
from openpose.op_utils import draw_kp2d_to_image,draw_reided_bboxes
from common.mincostflow import MinCostFlowReID

from mqtt_player.unity3d_mqtt_player import Unity3DMqttPlayer,MQTTPLAYER_CFG

from mchmr2.hmr_cfg import HMR_ENCODER
from mchmr2.hmr_encoder import HMREncoder

from pipeline.global_cfg import CFG_MVIEW
from pipeline.mview_x2ds_to_A import SingleX2dsReID,SimpleHMREncoder,HMRWrapper
from pipeline.pipeline_utils import ComputeSMPL24CameraTrajectory

In [6]:
##main view
op25b_x2ds0 = np.load("E:/4.test_videos/yunpeng_mview/01/video_0.npz",allow_pickle=True)["op25b"].item()
video0_fname = "E:/4.test_videos/yunpeng_mview/01/video_0.mp4"

##sub view
op25b_x2ds1 = np.load("E:/4.test_videos/yunpeng_mview/01/video_1.npz",allow_pickle=True)["op25b"].item()
video1_fname = "E:/4.test_videos/yunpeng_mview/01/video_1.mp4"

In [7]:
class MViewX2dsToA:
    def __init__(self,cfg=CFG_MVIEW):
        self.cfg = cfg.clone()
        self.main_camera = cfg.main_camera
        self.sub_camera = cfg.sub_camera
        #single hmr
        self.hmr_single = HMRWrapper(cfg)
        #fusion hmr
        self.hmr_fusioner = SimpleHMREncoder(cfg.fusion)
        
        #traj solver
        self.traj_solver = ComputeSMPL24CameraTrajectory(cfg.K,cfg.traj_offset)
        
        #cross and temporal matching
        self.cross_km = MinCostFlowReID()
        self.single_x2ds_reid = {}
        for i in cfg.cameras:
            self.single_x2ds_reid[i] = SingleX2dsReID(cfg)

    def crossview_pairing(self,f0,f1):
        """
        maping f0 to f1
        fx is NxD
        fx are tensor object
        """
        M = f0.shape[0]
        N = f1.shape[0]
        
        ##step1. extand features
        new_f0 = f0.unsqueeze(1).repeat(1,N,1)#MxNxD
        new_f1 = f1.unsqueeze(0).repeat(M,1,1)#MxNxD
        
        ##step3.KM assign
        cost_mat = torch.norm(new_f0-new_f1,dim=-1)
        assign = self.cross_km(cost_mat.cpu().numpy().astype(np.int))
        return assign
    
    def push(self,index,x2ds_dict):
        """
        x2ds_dict: key is camera name, value is Mx24x2
        return:
        @matched: 
            hids,M
            x2ds,MxCx24x2
            norm_x2ds,MxCx24x2
        @unmached:
            hids,N
            x2ds,Nx24x2
            hmr_return,Nx48
        """
        assert isinstance(x2ds_dict,dict)
        
        #step1.singleview forward
        N = x2ds_dict[self.main_camera].shape[0]
        main_hids = None
        reid_x2ds_dict = {}
        reid_humans_dict = {}
        reid_slice_dict = {}
        counter = 0
        for k in x2ds_dict:
            ret = self.single_x2ds_reid[k].push(index,x2ds_dict[k])##x2ds and hids
            if k==self.main_camera:
                main_hids = ret["hids"]
            reid_x2ds_dict[k]=ret["x2ds"]#tensor of Nx24x2
            reid_humans_dict[k] = ret["humans"]
            reid_slice_dict[k] = slice(counter,counter+ret["x2ds"].shape[0])
            counter += ret["x2ds"].shape[0]
        
        ##step2.hmr simple forward
        hmr_ret = self.hmr_single(torch.cat(list(reid_x2ds_dict.values())))
        norm_x2ds0 = hmr_ret["x"][reid_slice_dict[self.main_camera]]
        norm_x2ds1 = hmr_ret["x"][reid_slice_dict[self.sub_camera]]
        f0 = hmr_ret["y"][reid_slice_dict[self.main_camera]]
        f1 = hmr_ret["y"][reid_slice_dict[self.sub_camera]]
        
        #step2.cross matching
        cross_assign = self.crossview_pairing(f0[:,6:38],f1[:,6:38])
        
        
        #step3.match x2ds
        matched_dict = {"hids":[],
                        "x2ds":[],
                        "norm_x2ds":[]}
        unmatched_dict = {"hids":[],
                          "x2ds":[],
                          "hmrs":[]}
        for i in range(N):
            x0 = reid_x2ds_dict[self.main_camera][i]
            hid = main_hids[i]
            if i in cross_assign:
                mid = cross_assign[i][0] # matched index in subview
                x1 = reid_x2ds_dict[self.sub_camera][mid]#24x2
                matched_dict["hids"].append(hid)
                matched_dict["x2ds"].append(torch.stack([x0,x1]))
                matched_dict["norm_x2ds"].append(torch.stack([norm_x2ds0[i],norm_x2ds1[mid]]))
            else:
                unmatched_dict["hids"].append(hid)
                unmatched_dict["x2ds"].append(x0)
                unmatched_dict["hmrs"].append(f0[i])
        final_hids = matched_dict["hids"] + unmatched_dict["hids"]
        
        ##matched
        if len(matched_dict["hids"])>0:
            matched_dict["x2ds"] = torch.stack(matched_dict["x2ds"])
            matched_dict["norm_x2ds"] = torch.stack(matched_dict["norm_x2ds"])
        else:
            matched_dict = None
        ##unmatched
        if len(unmatched_dict["hids"])>0:
            unmatched_dict["x2ds"] = torch.stack(unmatched_dict["x2ds"])
            unmatched_dict["hmrs"] = torch.stack(unmatched_dict["hmrs"])
        else:
            unmatched_dict = None
        
        ##step4.matched x2ds to A
        final_A = []
        if matched_dict is not None:
            ret = self.hmr_fusioner.forward_hmr(matched_dict["norm_x2ds"])
            matched_A = self.hmr_fusioner.forward_smpl(ret["root_r6d"],ret["pose_r6d"])
            final_A.append(matched_A)
            
        if unmatched_dict is not None:
            ##step5.unmatched x2ds to A
            with torch.no_grad():
                root_r6d = unmatched_dict["hmrs"][:,:6]
                pose_r6d = self.hmr_fusioner.vposer.decode(unmatched_dict["hmrs"][:,6:38])["r6d"]
            unmatched_A = self.hmr_fusioner.forward_smpl(root_r6d,pose_r6d)
            final_A.append(unmatched_A)
        final_A = torch.cat(final_A)
        final_A = final_A.cpu().numpy()
        
        ##step5.compute traj
        trajs = self.traj_solver(reid_x2ds_dict[self.main_camera].numpy(),final_A[...,:3,-1])#Nx3
        for i in range(N):
            trajs[i,[0,2]] = reid_humans_dict[self.main_camera][i].traj_filter(index,trajs[i,[0,2]])
            final_A[i,:,:3,-1] += trajs[i].reshape(1,3)  #update traj in A 
        final_A = np.matmul(self.traj_solver.camera_to_world_R,final_A)
        return {"A":final_A,
                "hids":final_hids}

In [8]:
map_op25b_to_smpl24 = NPMapOP25bToSMPL24()
mview_x2ds_to_A = MViewX2dsToA(CFG_MVIEW)

>>>Loaded HMR from:../12.models/mchmr2/20200516-hmrzero-b4f2048s10-vposer0420.pth
>>>Loaded HMR from:../12.models/mchmr2/20200528-hmrzeofusion-x2b5f2048s5-vposer0420.pth
>>>Loaded VPoser from:../12.models/mchmr2/20200420-vposer1024-lt32b3.pth


In [9]:
window_name = "reID window"
cv2.namedWindow(window_name)
## video1
cap0 = cv2.VideoCapture(video0_fname)
h0 = int(cap0.get(cv2.CAP_PROP_FRAME_HEIGHT))
w0 = int(cap0.get(cv2.CAP_PROP_FRAME_WIDTH))
## video2
cap1 = cv2.VideoCapture(video1_fname)
h1 = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
w1 = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))

height = h0+h1
width = max(w0 , w1)
canvas = np.zeros((height, width, 3),dtype=np.uint8)
mqtt_player = Unity3DMqttPlayer(MQTTPLAYER_CFG)

for i in range(1000):
    is_valid0,frame0 = cap0.read()
    is_valid1,frame1 = cap1.read()
    if not is_valid0 or not is_valid1:
        break
        
    #step1.load data from each view
    if i in op25b_x2ds0 and i in op25b_x2ds1:
        ts = time.time()
        ##step1.load x2ds
        x2ds0 = map_op25b_to_smpl24(op25b_x2ds0[i])[...,:2] #Nx24x3
        x2ds1 = map_op25b_to_smpl24(op25b_x2ds1[i])[...,:2] #Mx24x3
        
        ret = mview_x2ds_to_A.push(i,{0:x2ds0,1:x2ds1})
        mqtt_player(ret["hids"],ret["A"].reshape(-1,384))
        ts = time.time()-ts
    
    canvas[:h0] = frame0
    canvas[h0:] = frame1
    new_canvas = cv2.resize(canvas,(800,900))
    cv2.imshow(window_name,new_canvas)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break
cv2.destroyAllWindows()